In [ ]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re
import winsound as ws
from konlpy.tag import Kkma,Twitter, Komoran
kkma = Kkma()

In [ ]:
path = 'C:/Users/mbnb8/Downloads/chromedriver.exe'

driver = webdriver.Chrome(path) 

driver.get('https://www.ndsl.kr/ndsl/search/list/article/articleSearchResultList.do?dbtype=&dbtype=JAKO&dbtype=JAKO&dbtype=&dbtype=&dbtype=JAKO&dbtype=CFKO&dbtype=&dbtype=&dbtype=JAKO&resultCount=10&prefixQuery=%28%28%3CLANG_PREFIX%3Acontains%3Akor%3E%29%29+%28%28%3CPUBYEAR_PREFIX%3Acontains%3A2019%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2018%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2017%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2016%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2015%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2014%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2013%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2012%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2011%3E%29%7C%28%3CPUBYEAR_PREFIX%3Acontains%3A2010%3E%29%29&filterValue=l_kor%40%40%ED%95%9C%EA%B5%AD%EC%96%B4+%7C%7Cp_2019%40%402019%7C%7Cp_2018%40%402018%7C%7Cp_2017%40%402017%7C%7Cp_2016%40%402016%7C%7Cp_2015%40%402015%7C%7Cp_2014%40%402014%7C%7Cp_2013%40%402013%7C%7Cp_2012%40%402012%7C%7Cp_2011%40%402011%7C%7Cp_2010%40%402010&collectionQuery=&showQuery=%EB%B9%84%EB%A7%8C&query=%EB%B9%84%EB%A7%8C&sortName=RANK&sortOrder=DESC&prefixCount=8810&pageForMoreList=10&detailEnv=1&detail_search_01__input_complex=&detailSearchApplyNum=01&svcCode1=SC&svcCode2=SC05&selectSeq=&colTypeByUser=&pageCode=PG04')

In [ ]:
title = []
abstract = []

In [ ]:
for nex in range(30,40):    
    if nex > 0:
        for page in range(4,14):
            for i in range(1,11):
                #초록이 없는 경우는 어떡할래?
                try:
                    driver.find_elements_by_xpath('''//*[@id="result_list"]/li['''+str(i)+''']/div[2]/ul/li[6]/a/span''')[0].click()
                except:
                    try :
                        driver.find_elements_by_xpath('''//*[@id="result_list"]/li['''+str(i)+''']/div[2]/ul/li[5]/a/span''')[0].click()
                    except:
                        continue
                    
                
            for i in range(10):
                abstract += [driver.find_elements_by_class_name("box_st1")[3*i].text]
                title += [driver.find_elements_by_class_name("list_con")[i].text]

            driver.find_elements_by_xpath('''//*[@id="srchFrm"]/div[5]/ul/li['''+str(page)+''']/a''')[0].click()    
            time.sleep(1)
    else :
        for page in range(2,13):
            driver.find_elements_by_xpath('''//*[@id="srchFrm"]/div[5]/ul/li['''+str(page)+''']/a''')[0].click()
            time.sleep(1)
            
            for i in range(1,11):
                    try:
                        driver.find_elements_by_xpath('''//*[@id="result_list"]/li['''+str(i)+''']/div[2]/ul/li[6]/a/span''')[0].click()
                    except IndexError:
                        driver.find_elements_by_xpath('''//*[@id="result_list"]/li['''+str(i)+''']/div[2]/ul/li[5]/a/span''')[0].click()
    
            for i in range(10):
                abstract += [driver.find_elements_by_class_name("box_st1")[3*i].text]
                title += [driver.find_elements_by_class_name("list_con")[i].text]
            
    

In [ ]:
title_real = []
abstract_real = []
for a, b in zip(title, abstract):
    title_real += [a.split('\n')[0].split('   ')[1]]
    abstract_real += [b.replace('초록\n','')]

del(title)
del(abstract)

In [ ]:
biman_abstract = [abstract_real[i] for i, x in enumerate(title_real) if '비만' in x]
biman_title = [x for i, x in enumerate(title_real) if '비만' in x]

In [ ]:
title_new = []
for ab in title_real:
    a = re.sub('\n',' ',ab) # \r\n 제거
    b = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',a) # 특수문자 제거
    title_new += [b]

del(title_real)

abs_new = []
for ab in abstract_real:
    a = re.sub('\n',' ',ab) # \r\n 제거
    b = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',a) # 특수문자 제거
    abs_new += [b]

del(abstract_real)

## POS-tagging : 명사, 고유 명사

In [ ]:
data = []
for ab in abs_new:
    pos = kkma.pos(ab)
    nouns = []
    for keyword, type in pos:
        if type=='NNG' or type == 'NNP':
            nouns.append(keyword)
    data += [nouns]

In [ ]:
pd.DataFrame({'word' : data}).to_csv('abstract4.csv',encoding='ms949')